In [ ]:
from core.sql_utils import *
import plotly.express as px

In [ ]:
engine = get_sqlalchemy_engine()
con = engine.connect()

with engine.connect() as connection:
    dbeaver_df = pd.read_sql(text("""SELECT * FROM vehicle_data vd
            join vehicle v
            on v.id = vd.vehicle_id
            join vehicle_model vm 
            on vm.id = v.vehicle_model_id
            join battery b
            on b.id = vm.battery_id
            join fleet f
            on f.id = v.fleet_id
            join oem o
            on o.id=vm.oem_id
            WHERE o.oem_name = 'tesla'
            ;"""), con)

dbeaver_df.shape

## graph

In [ ]:
dbeaver_df.vin.nunique()

In [ ]:
dbeaver_df.groupby('vin')['fleet_name'].first().value_counts()

In [ ]:
px.scatter(dbeaver_df, x='odometer', y='soh', color='battery_name', hover_data={'vin': True,
                                                                                'version': True})

In [ ]:
px.scatter(dbeaver_df[dbeaver_df['battery_name']=="BTX9"], x='odometer', y='soh', color='version', hover_data={'vin': True,
                                                                                'version': True})

In [ ]:
dbeaver_df[dbeaver_df['battery_name']=="BTX5"]

In [ ]:
info = dbeaver_df.groupby("vin", as_index=False).agg(
    soh_min=('soh', 'min'),
    soh_max=('soh', "max"),
    odometer=('odometer', 'max'),
)


# vin too high 

In [ ]:
vin_too_high = info[info['soh_min'] > 1].vin.unique()

In [ ]:
px.scatter(dbeaver_df[dbeaver_df['vin'].isin(vin_too_high)], x='odometer', y='soh', color='version', hover_data={'version': True})

In [ ]:
px.scatter(dbeaver_df[dbeaver_df['version']=='MTY29'], x='odometer', y='soh', color='vin')

## Vin too low

In [ ]:
vin_too_low = info[(info['soh_max'] < .95) & (info['odometer'] < 50000)].vin.unique()

In [ ]:
px.scatter(dbeaver_df[(dbeaver_df['vin'].isin(vin_too_low))], x='odometer', y='soh', color='version')

## Vin pb de croissance 

In [ ]:
px.scatter(dbeaver_df[dbeaver_df['vin'].isin(vin_too_low)], x='odometer', y='soh', color='vin')

In [ ]:
soh_diff = dbeaver_df.sort_values('odometer').groupby('vin', as_index=False).agg(
    soh_first=('soh', 'first'),
    soh_last=('soh', 'last')
).eval('soh_diff=soh_first-soh_last')

In [ ]:
soh_diff[soh_diff['soh_diff'] < 0].vin.unique()

In [ ]:
soh_diff[soh_diff['soh_diff'] < 0]